This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using [scipy](https://docs.scipy.org/doc/scipy/reference/).

In [27]:
# Import libraries
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
import matplotlib.pyplot as plt

This example has a series reaction $A \rightarrow B \rightarrow C$. The dataset consists of measures concentrations of A, B and C over time. The goal is to estimate the rate constants $k_1$ and $k_2$ for the two reactions.

In [3]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()

,t,ca,cb,cc
0,0.000,0.957,-0.031,-0.015
1,0.263,0.557,0.330,0.044
2,0.526,0.342,0.512,0.156
3,0.789,0.224,0.499,0.310
4,1.053,0.123,0.428,0.454


In [4]:
# function used in simulating the ODE that gives rates for given level of concentration at time t
def rate(t, c, p):
    
    # concentration
    ca = c[0]
    cb = c[1]
    cc = c[2]
    
    # rate constants
    k1 = p[0]
    k2 = p[1]
    
    # rate of formation of A, B, C
    dca = -k1 * ca 
    dcb = k1 * ca - k2 * cb
    dcc = k2 * cb
    
    dc = [dca, dcb, dcc]
    
    return dc

In [12]:
# function to calculate sum of squares for a given value of parameters
def ssq(p, data):
    ratefn = lambda t, c: rate(t, c, p)
    sol = solve_ivp(ratefn, [0.0, 5.0], [1.0, 0.0, 0.0], method = "BDF", t_eval = data.t.values)
    ssq_val = np.sum((sol.y[0, :] - data.ca.values)**2) + np.sum((sol.y[1, :] - data.cb.values)**2) + np.sum((sol.y[2, :] - data.cc.values)**2)
    return ssq_val

In [28]:
res = minimize(ssq, [0.1, 0.1], method = 'BFGS', args = (data_df,))

In [29]:
# Estimated parameters
p_sol = res.x
p_sol

array([2.0184544 , 0.99337149])

In [40]:
# covariance matrix estimate
n = data_df.shape[0] * 3 # number of data points
k = 2 # number of parameters
ssq_val = res.fun # residual sum of squares
cov_est = 2 * ssq_val / (n - k) * res.hess_inv
cov_est

array([[ 0.00240278, -0.00037099],
       [-0.00037099,  0.00033067]])